<a href="https://www.kaggle.com/code/yaaangzhou/playground-s3-e21-very-simple-approach?scriptVersionId=140741557" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s3e21/sample_submission.csv')

In [3]:
df = df[df.O2_1<20]
df = df[df.O2_2<50]
df = df[df.BOD5_5<40]
df = df[df.target<20]

In [4]:
col_to_drop = ['BOD5_1', 'NO3_2', 'O2_7', 'O2_6', 'BOD5_3', 'NH4_5', 'BOD5_2', 'NO2_5', 'BOD5_7', 'BOD5_5', 'BOD5_6', 'BOD5_4', 'NH4_1', 'NO3_1', 'NH4_3', 'O2_3', 'O2_5', 'NO2_1', 'NO3_4', 'NH4_2', 'NO2_7', 'NO2_6', 'NO3_7', 'NH4_4', 'NH4_7', 'NO2_4', 'NO3_3', 'NO3_5', 'NO3_6', 'NO2_3']

for i in df.columns:
    if i in col_to_drop:
        df[i]=0

In [5]:
df.to_csv('submission.csv',index=False)
df

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
0,0,8.59,7.500,9.000,0,9.265,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,9.10,13.533,40.900,0,9.265,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,8.21,3.710,5.420,0,9.265,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,8.39,8.700,8.100,0,9.200,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,8.07,8.050,8.650,0,9.265,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3495,8.08,6.250,8.300,0,9.265,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3496,3496,8.09,6.630,6.630,0,7.600,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3497,3497,9.95,8.367,8.433,0,6.170,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3498,3498,9.52,10.000,6.630,0,9.265,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
